_Author:_ Eric Bruning, Texas Tech University, 11 June 2021

## The joy of the always-transformable 3D earth-centered, earth-fixed coordinate

The basic principle we'll exploit here is that geodetic latitude, longitude, and altitude are a proper 3D coordinate basis referenced with repsect to an ellipsoid. Those coordinates can be mapped, with no approximations, forward and backward to any other 3D coordinate basis, including a cartesian system located at the center of the earth, and which rotates with the earth.

## Setup

In [1]:
import numpy as np
import pyproj as proj4

def centers_to_edges(x):
    xedge=np.zeros(x.shape[0]+1)
    xedge[1:-1] = (x[:-1] + x[1:])/2.0
    dx = np.mean(np.abs(xedge[2:-1] - xedge[1:-2]))
    xedge[0] = xedge[1] - dx
    xedge[-1] = xedge[-2] + dx
    return xedge


In [2]:
def get_proj(ctr_lon, ctr_lat):
    # Define a WGS84 earth (just to show doing so for an arbitrary globe)
    earth_major, earth_minor =  6378137.0, 6356752.3142
    hou_ctr_lat, hou_ctr_lon = ctr_lat, ctr_lon
    # x, y = np.meshgrid(x,y)
    stereo = proj4.crs.CRS(proj='stere', a=earth_major, b=earth_minor,
                         lat_0=hou_ctr_lat, lon_0=hou_ctr_lon)
    lla = proj4.crs.CRS(proj='latlong', a=earth_major, b=earth_minor)
    ecef = proj4.crs.CRS(proj='geocent', a=earth_major, b=earth_minor)

    return lla, ecef, stereo

def get_grid_ctr_edge(dlat=0.1, dlon=0.1, dalt=1000.0, ctr_lon=0.0, ctr_lat=0):
    hou_ctr_lat, hou_ctr_lon = ctr_lat, ctr_lon
    nlon, nlat = 50, 50
    nalt = 50
    lon = dlon*(np.arange(nlon, dtype='float') - nlon/2) + dlon/2
    lat = dlat*(np.arange(nlat, dtype='float') - nlat/2) + dlat/2
    alt = dalt*(np.arange(nalt, dtype='float') - nalt/2) + dalt/2
    lon += hou_lon
    lat += hou_lat
    alt += hou_alt - alt.min()
    lon_edge = centers_to_edges(lon)
    lat_edge = centers_to_edges(lat)
    alt_edge = centers_to_edges(alt)
    return lon, lat, alt, lon_edge, lat_edge, alt_edge



Set up coordinate systems and transformers between each

In [3]:
hou_lat, hou_lon, hou_alt = 29.4719, -95.0792, 0.0

lla, ecef, stereo = get_proj(hou_lon, hou_lat)
lon, lat, alt, lon_edge, lat_edge, alt_edge = get_grid_ctr_edge(ctr_lon=hou_lon, ctr_lat=hou_lat)

lla_to_ecef = proj4.Transformer.from_crs(lla, ecef)

Where is our center location in earth-centered, earth-fixed cartesian coordinates? _ECEF X, Y, and Z have nothing to do with east, north, and up_. Rather, their unit vectors point from the earth center toward z=north pole, x=(0°N,0°E) and y=(0°N, 90°E), the right handed vector with x and z. See the [WGS84 implementation manual, Appendix B.](https://www.icao.int/safety/pbn/Documentation/EUROCONTROL/Eurocontrol%20WGS%2084%20Implementation%20Manual.pdf)

It'll be a big number in kilometers, in each vector component, and absolute distance from Earth's center should be close to 6370 km. 

In [4]:
Xctr_ecef, Yctr_ecef, Zctr_ecef = lla_to_ecef.transform(hou_lon, hou_lat, hou_alt)

print(Xctr_ecef/1000, Yctr_ecef/1000, Zctr_ecef/1000)

def distance_3d(x,y,z):
    """ Given x, y, and z distances from (0,0,0), find the total distance along a ray from the origin"""
    return(np.sqrt(x*x + y*y +z*z))

hou_distance_from_earth_center = distance_3d(Xctr_ecef/1000, Yctr_ecef/1000, Zctr_ecef/1000)
print(hou_distance_from_earth_center)

-492.00206497852935 -5535.470031072471 3119.543449811821
6372.9934749777085


Next we'll create a 3D mesh of lat, lon, alt that corresponds to regular geodetic coordinates. These arrays are redundant along two of their axes, of course - that's what it means to be "regular."

We will use the grid edges (formally, the node positions where the grid box boundaries intersect) instead of the grid cell centers, since we want to think of a mesh that (in this case) wraps around the earth elliposid, with some distortion of the shape of those cell interiors. Specifically, the great circle distance along the elliposid along lontitude changes substantially with latitude. There is also a subtle expansion of the cells in altitude, in a sort of conical expansion.

In [5]:
lon_edge_3d, lat_edge_3d, alt_edge_3d,  = np.meshgrid(lon_edge, lat_edge, alt_edge, indexing='ij')
lon_3d, lat_3d, alt_3d,  = np.meshgrid(lon, lat, alt, indexing='ij')

print(lon_edge_3d[:,0,0])
print(lat_edge_3d[0,:,0])
print(alt_edge_3d[0,0,:])

[-97.5792 -97.4792 -97.3792 -97.2792 -97.1792 -97.0792 -96.9792 -96.8792
 -96.7792 -96.6792 -96.5792 -96.4792 -96.3792 -96.2792 -96.1792 -96.0792
 -95.9792 -95.8792 -95.7792 -95.6792 -95.5792 -95.4792 -95.3792 -95.2792
 -95.1792 -95.0792 -94.9792 -94.8792 -94.7792 -94.6792 -94.5792 -94.4792
 -94.3792 -94.2792 -94.1792 -94.0792 -93.9792 -93.8792 -93.7792 -93.6792
 -93.5792 -93.4792 -93.3792 -93.2792 -93.1792 -93.0792 -92.9792 -92.8792
 -92.7792 -92.6792 -92.5792]
[26.9719 27.0719 27.1719 27.2719 27.3719 27.4719 27.5719 27.6719 27.7719
 27.8719 27.9719 28.0719 28.1719 28.2719 28.3719 28.4719 28.5719 28.6719
 28.7719 28.8719 28.9719 29.0719 29.1719 29.2719 29.3719 29.4719 29.5719
 29.6719 29.7719 29.8719 29.9719 30.0719 30.1719 30.2719 30.3719 30.4719
 30.5719 30.6719 30.7719 30.8719 30.9719 31.0719 31.1719 31.2719 31.3719
 31.4719 31.5719 31.6719 31.7719 31.8719 31.9719]
[ -500.   500.  1500.  2500.  3500.  4500.  5500.  6500.  7500.  8500.
  9500. 10500. 11500. 12500. 13500. 14500. 1550

In [6]:
zi_at_ground = 0 # actually the edge 0.5 dz below the ellipsoid (500 m for 1 km spacing).
zi_top = -1 # actually the edge 0.5 dz below the ellipsoid (500 m for 1 km spacing).
yi_north_edge = -1
yi_south_edge = 0
xi_east_edge = -1
xi_west_edge = 0

The indexing order above corresponds to variation along lon, lat, alt, or east, north, and up. These are x, y, z in the local reference frame tangent to the ground as they are commonly used in meteorology.

Now, we convert to ECEF coordinates. These will be ordinary distances in meters.

In [7]:
Xecef_edge_3d, Yecef_edge_3d, Zecef_edge_3d = lla_to_ecef.transform(lon_edge_3d, lat_edge_3d, alt_edge_3d)

In [8]:
print(Xecef_edge_3d[:,0,0])
print(Yecef_edge_3d[0,:,0])
print(Zecef_edge_3d[0,0,:])

[-750208.21416856 -740366.62826717 -730522.78708064 -720676.72059501
 -710828.45880311 -700978.03170444 -691125.46930512 -681270.80161776
 -671414.05866139 -661555.27046137 -651694.46704925 -641831.67846277
 -631966.93474569 -622100.26594773 -612231.70212447 -602361.27333726
 -592489.00965316 -582614.94114478 -572739.09789025 -562861.5099731
 -552982.20748217 -543101.22051154 -533218.57916041 -523334.31353301
 -513448.45373854 -503561.02989103 -493672.07210929 -483781.6105168
 -473889.67524163 -463996.29641633 -454101.50417784 -444205.32866741
 -434307.80003052 -424408.94841675 -414508.80397972 -404607.39687698
 -394704.75726995 -384800.91532377 -374895.90120727 -364989.74509284
 -355082.47715635 -345174.12757705 -335264.7265375  -325354.30422345
 -315442.89082376 -305530.51653033 -295617.21153795 -285703.00604428
 -275787.9302497  -265872.01435726 -255955.28857256]
[-5638161.54111362 -5633171.75231946 -5628164.8240174  -5623140.77079247
 -5618099.60728132 -5613041.34817216 -5607966.00

## Tests

Since our grid was regular in latitude, longitude and altitude, we should observe
1. a difference in the east-west spacing as we move north-south
2. not much difference in north-south spacing as we move east-west (only that due to the ellipsoid's oblateness)
3. larger spacing at higher altitudes
4. No difference in altitude spacing regardless of position

The distances along the edges use all three ECEF coordinates, since they do not vary regularly with lon,lat,alt.

We are going to calculate grid spacings as though the earth is locally flat over the 0.1 deg spacing of our grid. Strictly speaking, the edges of our grid boxes make a chord with the earth's curvature.

1. Here is where we expect the largest difference: it's about 500 m for a 0.1 deg spacing in lat and lon across a 5 deg span.

In [9]:
# Calculate all E-W spacings. We move east, along a line of latitude, and calculate for all longitudes and altitudes
left_edges = slice(None, -1)
right_edges = slice(1, None)
ew_spacing_X = (Xecef_edge_3d[right_edges, :, :] - Xecef_edge_3d[left_edges, :, :])
ew_spacing_Y = (Yecef_edge_3d[right_edges, :, :] - Yecef_edge_3d[left_edges, :, :])
ew_spacing_Z = (Zecef_edge_3d[right_edges, :, :] - Zecef_edge_3d[left_edges, :, :])

ew_distances = distance_3d(ew_spacing_X, ew_spacing_Y, ew_spacing_Z)

print("Difference in east-west spacing as we move north-south, along west edge")
print(ew_distances[xi_west_edge, yi_north_edge, zi_at_ground])
print(ew_distances[xi_west_edge, yi_south_edge, zi_at_ground])

print("Difference in east-west spacing as we move north-south, along east edge")
print(ew_distances[xi_east_edge, yi_north_edge, zi_at_ground])
print(ew_distances[xi_east_edge, yi_south_edge, zi_at_ground])

print("There is a change in east-west distance due to the narrowing of lines of longitude toward the poles.")
print("The pairs are identifical no matter their longitudintal position, as it should be geometrically")

Difference in east-west spacing as we move north-south, along west edge
9451.453419873573
9927.17590991852
Difference in east-west spacing as we move north-south, along east edge
9451.453419873615
9927.175909918531
There is a change in east-west distance due to the narrowing of lines of longitude toward the poles.
The pairs are identifical no matter their longitudintal position, as it should be geometrically


2. Here is the difference due to oblateness: about 10 m for a 0.1 deg lat lon spacing across a 5 deg span.

In [10]:
# Calculate all N-S spacings. We move north, along a line of longitude, and calculate for all longitudes and altitudes
# This is indexing like a[1:] - a[:-1]
left_edges = slice(None, -1)
right_edges = slice(1, None)
ns_spacing_X = (Xecef_edge_3d[:, right_edges, :] - Xecef_edge_3d[:, left_edges, :])
ns_spacing_Y = (Yecef_edge_3d[:, right_edges, :] - Yecef_edge_3d[:, left_edges, :])
ns_spacing_Z = (Zecef_edge_3d[:, right_edges, :] - Zecef_edge_3d[:, left_edges, :])

ns_distances = distance_3d(ns_spacing_X, ns_spacing_Y, ns_spacing_Z)

print("Difference in north-south spacing as we move east-west, along south edge")
print(ns_distances[xi_east_edge, yi_south_edge, zi_at_ground])
print(ns_distances[xi_west_edge, yi_south_edge, zi_at_ground])

print("Difference in north-south spacing as we move east-west, along north edge")
print(ns_distances[xi_east_edge, yi_north_edge, zi_at_ground])
print(ns_distances[xi_west_edge, yi_north_edge, zi_at_ground])

print("The north-south spacing is not identical along the northern and southern edges, since the earth is oblate.")
print("There is no difference in in the north-south spacing along each edge, as expected from geometry.")

Difference in north-south spacing as we move east-west, along south edge
11079.512436728495
11079.512436728604
Difference in north-south spacing as we move east-west, along north edge
11087.670300165752
11087.67030016583
The north-south spacing is not identical along the northern and southern edges, since the earth is oblate.
There is no difference in in the north-south spacing along each edge, as expected from geometry.


3. Here is the difference in east west and north south spacing as a function of altitude. There's about a 70 m increase in e-w spacing (at this latitude) at the top of the column, and a 100 m increase in n-s spacing, for a 50 km depth (troposphere and stratosphere).

In [11]:
# Calculate all N-S spacings. We move north, along a line of longitude, and calculate for all longitudes and altitudes
# This is indexing like a[1:] - a[:-1]


print("Spacing at ground and top in east-west direction, northwest corner")
print(ew_distances[xi_west_edge, yi_north_edge, zi_at_ground])
print(ew_distances[xi_west_edge, yi_north_edge, zi_top])

print("Spacing at ground and top in east-west direction, southwest corner")
print(ew_distances[xi_west_edge, yi_south_edge, zi_at_ground])
print(ew_distances[xi_west_edge, yi_south_edge, zi_top])

print("Spacing at ground and top in north-south direction, northwest corner")
print(ns_distances[xi_west_edge, yi_north_edge, zi_at_ground])
print(ns_distances[xi_west_edge, yi_north_edge, zi_top])

print("Spacing at ground and top in north-south direction, southwest corner")
print(ns_distances[xi_west_edge, yi_south_edge, zi_at_ground])
print(ns_distances[xi_west_edge, yi_south_edge, zi_top])



Spacing at ground and top in east-west direction, northwest corner
9451.453419873573
9525.482238923145
Spacing at ground and top in east-west direction, southwest corner
9927.17590991852
10004.950308439165
Spacing at ground and top in north-south direction, northwest corner
11087.67030016583
11174.936751690027
Spacing at ground and top in north-south direction, southwest corner
11079.512436728604
11166.778888252507


4. Here is the difference in vertical spacing as a function of horizontal position. No difference.

In [12]:
# Calculate all N-S spacings. We move north, along a line of longitude, and calculate for all longitudes and altitudes
# This is indexing like a[1:] - a[:-1]
left_edges = slice(None, -1)
right_edges = slice(1, None)
ud_spacing_X = (Xecef_edge_3d[:, :, right_edges] - Xecef_edge_3d[:, :, left_edges])
ud_spacing_Y = (Yecef_edge_3d[:, :, right_edges] - Yecef_edge_3d[:, :, left_edges])
ud_spacing_Z = (Zecef_edge_3d[:, :, right_edges] - Zecef_edge_3d[:, :, left_edges])

ud_distances = distance_3d(ud_spacing_X, ud_spacing_Y, ud_spacing_Z)

print("Difference in vertical spacing as we move up, southeast corner")
print(ud_distances[xi_east_edge, yi_south_edge, zi_at_ground])
print(ud_distances[xi_east_edge, yi_south_edge, zi_top])

print("Difference in vertical spacing as we move up, northeast corner")
print(ud_distances[xi_east_edge, yi_north_edge, zi_at_ground])
print(ud_distances[xi_east_edge, yi_north_edge, zi_top])

print("Difference in vertical spacing as we move up, northwest corner")
print(ud_distances[xi_west_edge, yi_north_edge, zi_at_ground])
print(ud_distances[xi_west_edge, yi_north_edge, zi_top])

print("Difference in vertical spacing as we move up, southwest corner")
print(ud_distances[xi_west_edge, yi_south_edge, zi_at_ground])
print(ud_distances[xi_west_edge, yi_south_edge, zi_top])


print("There is no difference in the vertical spacing, as it should be given the definition of our grid.")

Difference in vertical spacing as we move up, southeast corner
1000.000000000558
1000.0000000005568
Difference in vertical spacing as we move up, northeast corner
1000.0000000000877
999.9999999998411
Difference in vertical spacing as we move up, northwest corner
999.999999999273
999.9999999990395
Difference in vertical spacing as we move up, southwest corner
1000.0000000005921
999.9999999997692
There is no difference in the vertical spacing, as it should be given the definition of our grid.


## Summary

For a regular latitude, longitude grid, at the latitude of Houston:

- The largest difference is in the east-west spacing with north-south position, about 500 m / 10000 m
- The difference due to oblateness in n-s spacing with n-s position is 50x smaller, about 10 m / 10000 m
- The difference in spacing as a function of altitude is in between the above, and surprisingly large: 100 m / 10000 m
- Altitude spacing remains constant.

It is also a hypothesis that the rate of change of these spacings with position on the earth's surface (in the limit of small dx, dy) are related to the proj4 map factors.

The calculations above are easily adjusted to try other locations and grid spacings.

**Extension to map projections and model grid coordinates**

One could also compare the distances calculate in the exercise above to the stereographic x, y coordinate distances. Note we already defined the necessary stereographic coordinate system …

The same process as above could be done to convert a (for example) stereographic model grid to ECEF, from which the exact volumes could be calculated. Define a new proj4.Transformer.from_crs(stereo, lla), convert a meshgrid of 2D model x, y coords to (lat, lon), and replicate the 2D lat lon over the number of sigma coordinates to get 3D lon, lat grids. The 3D alt grid can be calcualted from the height information of each model sigma level at each model grid point. Then convert to ECEF!

## Bonus: grid volumes

These are the exact volumes in m$^3$ of our 3D mesh. 

While the volumes are not cubes, they are (I think) guaranteed to be convex since the faces are all planar and the postions are monotonic. So we can use the convex hull. We also try an approximate calculation using the simple spacings.

For a 0.1 deg grid that's about 10 km horizonal, and with 1 km vertical spacing we should have volumes of about 10x10x1=100 km$^3$

In [13]:
import numpy as np
from scipy.spatial import ConvexHull

# Need an Mx8x3 array for M grid boxes with 8 corners.
# WSB, ESB, 
# ENB, WNB, 
# WST, EST, 
# ENT, WNT (east west north south bottom top)
# S,W,B are :-1
# N,E,T are 1:
x_corners = [Xecef_edge_3d[:-1,:-1,:-1], Xecef_edge_3d[ 1:,:-1,:-1],
             Xecef_edge_3d[ 1:, 1:,:-1], Xecef_edge_3d[:-1, 1:,:-1],
             Xecef_edge_3d[:-1,:-1, 1:], Xecef_edge_3d[ 1:,:-1, 1:],
             Xecef_edge_3d[ 1:, 1:, 1:], Xecef_edge_3d[:-1, 1:, 1:],]
y_corners = [Yecef_edge_3d[:-1,:-1,:-1], Yecef_edge_3d[ 1:,:-1,:-1],
             Yecef_edge_3d[ 1:, 1:,:-1], Yecef_edge_3d[:-1, 1:,:-1],
             Yecef_edge_3d[:-1,:-1, 1:], Yecef_edge_3d[ 1:,:-1, 1:],
             Yecef_edge_3d[ 1:, 1:, 1:], Yecef_edge_3d[:-1, 1:, 1:],]
z_corners = [Zecef_edge_3d[:-1,:-1,:-1], Zecef_edge_3d[ 1:,:-1,:-1],
             Zecef_edge_3d[ 1:, 1:,:-1], Zecef_edge_3d[:-1, 1:,:-1],
             Zecef_edge_3d[:-1,:-1, 1:], Zecef_edge_3d[ 1:,:-1, 1:],
             Zecef_edge_3d[ 1:, 1:, 1:], Zecef_edge_3d[:-1, 1:, 1:],]


# Get an Mx8 array
x_corner_points = np.vstack([a.flatten() for a in x_corners])
y_corner_points = np.vstack([a.flatten() for a in y_corners])
z_corner_points = np.vstack([a.flatten() for a in z_corners])
point_stack = np.asarray((x_corner_points,y_corner_points,z_corner_points)).T

volumes = np.fromiter((ConvexHull(polygon).volume for polygon in point_stack), 
            dtype=float, count=point_stack.shape[0])
volumes.shape=lon_3d.shape
print("Convex hull min, max volumes")
print(volumes.min()/(1e3**3))
print(volumes.max()/(1e3**3))

# Approximate version. Shapes are (50,51,51) (51,50,51) (51,51,50) to start.
# There are actually four E-W distances at the S, N, bottom, and top edges of each box,
# and so on.
ew_mean = (ew_distances[:,1:,1:]+ew_distances[:,1:,:-1]+ew_distances[:,:-1,1:]+ew_distances[:,:-1,:-1])/4
ns_mean = (ns_distances[1:,:,1:]+ns_distances[1:,:,:-1]+ns_distances[:-1,:,1:]+ns_distances[:-1,:,:-1])/4
ud_mean = (ud_distances[1:,1:,:]+ud_distances[1:,:-1,:]+ud_distances[:-1,1:,:]+ud_distances[:-1,:-1,:])/4

volumes_approx = ew_mean * ns_mean * ud_mean
print("Approximate min, max volumes")
print(volumes_approx.min()/(1e3**3))
print(volumes_approx.max()/(1e3**3))

Convex hull min, max volumes
104.86771219542136
111.6561359538487
Approximate min, max volumes
104.86779184139247
111.65622075874296


It turns out the approximate volume calculation is quite good, without all the expense of the convex hull calculation!

## Regularizing lat-lon

Practically speaking, datasets might contain angles that are 0 to 360 longitude or various other departures from -180 to 180. No doubt we'd even find a -720 to -360 somewhere in the wild…

In [14]:
weird_lons = np.arange(-360, 360, 30)
weird_lats = np.zeros_like(weird_lons)

Proj won't regularize it for us with an identity transformation:

In [15]:
lla_to_lla = proj4.Transformer.from_crs(lla, lla)
print(lla_to_lla.transform(weird_lons, weird_lats))

(array([-360., -330., -300., -270., -240., -210., -180., -150., -120.,
        -90.,  -60.,  -30.,    0.,   30.,   60.,   90.,  120.,  150.,
        180.,  210.,  240.,  270.,  300.,  330.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]))


But we do get that behavior if we're willing to bear a computational penalty of a round-trip through ECEF:

In [16]:
ecef_to_lla = proj4.Transformer.from_crs(ecef, lla)
print(ecef_to_lla.transform(*lla_to_ecef.transform(weird_lons, weird_lats)))

(array([   0.,   30.,   60.,   90.,  120.,  150., -180., -150., -120.,
        -90.,  -60.,  -30.,    0.,   30.,   60.,   90.,  120.,  150.,
        180., -150., -120.,  -90.,  -60.,  -30.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.]))


This solution gives us a predictably-bounded set of coordinates, to which we could predictably apply other logic for shifting the data with respect to a center longitude of interest.